# Step 1 Creating and formating the dataframe

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#GET DATA SECTION#
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

In [2]:
#check if scrape was successful, but didn't run this in this instance as it's a 1 mile long script!
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XfMOBwpAAEQAACbjuCMAAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":930529633,"wgRevisionId":930529633,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [3]:
#using soup object, I used the iterate function to cycle through the table "".wikitable" to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
#check how big the DataFrame is
canada_df.count()

Postcode         287
Borough          287
Neighbourhood    287
dtype: int64

In [5]:
#PROCESS DATA SECTION#
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#To do this, I overwrite the df called "canada_df", and this time, remove all cells with "Not assigned" from the "Borough" column.
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [6]:
#More than one neighborhood can exist in one postal code area.
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postcode if it isn't already
if(canada_df.index.name != 'Postcode'):
    canada_df = canada_df.set_index('Postcode')
    
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [7]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
canada_df['Neighbourhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [8]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
canada_df.shape


(103, 2)

# Step 2 - GEOSPACIAL DATA SECTION
Using the geocoder package was too slow, so in this section, I use the .csv file


In [9]:
#get data from file and name it geospace. I've been told it's in .csv format so I read in csv
url="http://cocl.us/Geospatial_data"
geospace = pd.read_csv(url)

In [10]:
#check if data loaded successfully with .head
geospace.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
#rename columns and set the index to be Postcode
geospace.columns = ["Postcode", "Latitude", "Longitude"]
if(geospace.index.name != 'Postcode'):
    geospace = geospace.set_index('Postcode')
    
geospace.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [12]:
#merge
canada_df = canada_df.join(geospace)
canada_df.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [13]:
#to get it like how it's shown in the test
canada_df.head(11)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494
M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937


# Step 3 - Exploring Toronto
This involves loading up all the correct dependencies, following our example from the NY assignment.
My end product is an interactive map that shows each neighbourhood, and distinguishes each by their by a different colour. 

In [69]:
# here is just to check more of the dataset that we have, seeing more of the boroughs and neighbourhoods
canada_df

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494
M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937


In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         704 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.0-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

In [23]:
#to get a better idea of what our data holds, I check how many boroughs and neighbourhoods we have. I find I have 11 boroughs
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(canada_df['Borough'].unique()),
        canada_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Canada.

In [33]:
# all the data is in Toronto, so I find the geolocation so I can make this as the default position for my folium map
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


I now plot all the data onto the map with blue circles

In [48]:
# create map of Canada using latitude and longitude values
map_Canada = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(canada_df['Latitude'], canada_df['Longitude'], canada_df['Borough'], canada_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada)  
    
map_Canada

Knowing all the Borough names, I make each borough its own dataset to I can select them later to have different colours in my map

In [71]:
Borough1 = canada_df[canada_df.Borough == "North York"]
Borough2 = canada_df[canada_df.Borough == "Downtown Toronto"]
Borough3 = canada_df[canada_df.Borough == "Queen's Park"]
Borough4 = canada_df[canada_df.Borough == "Scarborough"]
Borough5 = canada_df[canada_df.Borough == "East York"]
Borough6 = canada_df[canada_df.Borough == "Etobicoke"]
Borough7 = canada_df[canada_df.Borough == "York"]
Borough8 = canada_df[canada_df.Borough == "West Toronto"]
Borough9 = canada_df[canada_df.Borough == "East Toronto"]
Borough10 = canada_df[canada_df.Borough == "Central Toronto	"]
Borough11 = canada_df[canada_df.Borough == "Mississauga"]

With my data ready, I use the same code from the NY assignment, but change the dataframe to my own dataframes, and change the colours for each borough

In [75]:
# create map of Canada using latitude and longitude values
map_Canada2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map North York
for lat, lng, borough, Neighbourhood in zip(Borough1['Latitude'], Borough1['Longitude'], Borough1['Borough'], Borough1['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2)  

# add markers to map Downtownn Toronto
for lat, lng, borough, Neighbourhood in zip(Borough2['Latitude'], Borough2['Longitude'], Borough2['Borough'], Borough2['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map Scarborough
for lat, lng, borough, Neighbourhood in zip(Borough3['Latitude'], Borough3['Longitude'], Borough3['Borough'], Borough3['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map Queen's Park
for lat, lng, borough, Neighbourhood in zip(Borough4['Latitude'], Borough4['Longitude'], Borough4['Borough'], Borough4['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='darkred',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map East York
for lat, lng, borough, Neighbourhood in zip(Borough5['Latitude'], Borough5['Longitude'], Borough5['Borough'], Borough5['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map Etobicoke
for lat, lng, borough, Neighbourhood in zip(Borough6['Latitude'], Borough6['Longitude'], Borough6['Borough'], Borough6['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map York
for lat, lng, borough, Neighbourhood in zip(Borough7['Latitude'], Borough7['Longitude'], Borough7['Borough'], Borough7['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='beige',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map West Toronto
for lat, lng, borough, Neighbourhood in zip(Borough8['Latitude'], Borough8['Longitude'], Borough8['Borough'], Borough8['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map East Toronto
for lat, lng, borough, Neighbourhood in zip(Borough9['Latitude'], Borough9['Longitude'], Borough9['Borough'], Borough9['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='darkgreen',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map Central Toronto
for lat, lng, borough, Neighbourhood in zip(Borough10['Latitude'], Borough10['Longitude'], Borough10['Borough'], Borough10['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='lightgreen',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
# add markers to map Mississauga
for lat, lng, borough, Neighbourhood in zip(Borough11['Latitude'], Borough11['Longitude'], Borough11['Borough'], Borough11['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='cadetblue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Canada2) 
    
map_Canada2